<a href="https://colab.research.google.com/github/Jornay/AC1-Teo-POO/blob/master/Parser_in_compilers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import re

T_KEYWORD = "keyword"
T_OP = "op"
T_INT = "int"
T_STRING = "string"
T_ID = "id"
T_EOF = "eof"

class Token():

    def __init__(self, tipo, valor=None):
        self.tipo = tipo
        self.valor = valor

    def __str__(self):
        return '<%s %s>' % (self.tipo, self.valor)


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


def afd_int(token):
    try:
        token = int(token)
        return True
    except:
        return False

def afd_string(token):
    if token[0] == '"' and token[-1] == '"':
        if '"' not in token[1:-1]:
            return True
        else:
            raise ValueError('Aspas em um local inesperado.')
    else:
        return False

def afd_identificador(token):
    regex = re.compile('[a-zA-Z0-9_]+')
    r = regex.match(token)
    if r is not None:
        if r.group() == token:
            return True
        else:
            return False
    else:
        return False

def afd_principal(token):

    if token == "init":
        return Token(T_KEYWORD, 'init')

    elif token in "=+*/":
        return Token(T_OP, token)

    elif afd_int(token):
        return Token(T_INT, token)

    elif afd_string(token):
        return Token(T_STRING, token)

    elif afd_identificador(token):
        return Token(T_ID, token)

    else:
        raise ValueError('Valor inesperado')

    return None


class Parser():

    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = -1
        self.token_atual = None

        self.proximo()


    def proximo(self):
        self.pos += 1

        if self.pos >= len(self.tokens):
            self.token_atual = Token(T_EOF)
        else:
            self.token_atual = self.tokens[self.pos]

        print(self.token_atual)
        return self.token_atual


    def erro(self):
        raise Exception('Erro de sintaxe.')


    def use(self, tipo, valor=None):

        if self.token_atual.tipo != tipo:
            self.erro()
        elif valor is not None and self.token_atual.valor != valor:
            self.erro()
        else:
            self.proximo()


    def statement(self):
        """
        statement ::= <id> <op => expr
        """
        #Aqui é verificado se as operações fonecidadas pelo usuário
        #não contem erro de sintaxe
        self.use(T_ID);
        self.use(T_OP);
        self.expr();
        pass


    def expr(self):
        """
        expr ::= term ( <op +> | <op -> term )*
        """

        self.term()
        while self.token_atual.tipo == T_OP and self.token_atual.valor in ['+','-', '*', '/']:
            self.use(T_OP)
            self.term()


    def term(self):
        """
        term ::= <id> | <int>
        """

        if self.token_atual.tipo == T_INT:
            self.use(T_INT)
        elif self.token_atual.tipo == T_ID:
            self.use(T_ID)
        else:
            self.erro()


##############################################################################

arquivo = open('codigo.x','r')
ln = 1

tokens = []

for l in arquivo.readlines():

    # analisador lexico

    l = l.replace('\n','') # remove a quebra de linha

    for token in l.split():
        try:
            tokens.append(afd_principal(token))
        except Exception as e:
            print(tokens)
            print(str(e) + " na posição %i da linha %i" % (l.index(token), ln))
            raise StopExecution
    ln += 1

print([str(t) for t in tokens])

# analisador sintatico

parser = Parser(tokens)
parser.statement()

['<id a>', '<op *>', '<id b>', '<id A>', '<op =>', '<int 54>']
<id a>
<op *>
<id b>
<id A>
